<a href="https://www.kaggle.com/code/bhavkaur/hyper-parameter-optimizatoin?scriptVersionId=134791502" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **All Techniques of Hyper Parameter Optimization**
- GridSearchCV
- RandomizedSearchCV
- Bayesian Optimization -Automate Hyperparameter Tuning (Hyperopt)
- Sequential Model Based Optimization (Tuning a scikit-learn estimator with skopt)
- Optuna- Automate Hyperparameter Tuning
- Genetic Algorithms (TPOT Classifier)

### Why do we require hyper parameter tuning?
Hyperparameters directly control model structure, function, and performance. Hyperparameter tuning allows data scientists to tweak model performance for optimal results. This process is an essential part of machine learning, and choosing appropriate hyperparameter values is crucial for success.

In [36]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
import pandas as pd
df=pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Doing the below strp to remove any '0' values in 'Glucose' and similarly for 'Insulin'

In [38]:
import numpy as np
df['Glucose']=np.where(df['Glucose']==0, df['Glucose'].median(), df['Glucose'])
df['Insulin']=np.where(df['Insulin']==0, df['Insulin'].median(), df['Insulin'])
df['SkinThickness']=np.where(df['SkinThickness']==0, df['SkinThickness'].median(), df['SkinThickness'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


### Do we require feature scaling if we are using RandomForest?
### No, because RandomForest works on DecissionTree (makes branches).

In [39]:
X = df.drop('Outcome', axis=1)
y=df['Outcome']

In [40]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72,35.0,30.5,33.6,0.627,50
1,1,85.0,66,29.0,30.5,26.6,0.351,31
2,8,183.0,64,23.0,30.5,23.3,0.672,32
3,1,89.0,66,23.0,94.0,28.1,0.167,21
4,0,137.0,40,35.0,168.0,43.1,2.288,33


In [41]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

# Train Test Split

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.20, random_state=33)

In [43]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
pred = rf.predict(X_test)

In [44]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [45]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, pred))
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

[[357  48]
 [103 107]]
0.7544715447154472
              precision    recall  f1-score   support

           0       0.78      0.88      0.83       405
           1       0.69      0.51      0.59       210

    accuracy                           0.75       615
   macro avg       0.73      0.70      0.71       615
weighted avg       0.75      0.75      0.74       615



### The main parameters used by a RandomForestClassifier are:
- criterion = the funciton is used to evaluate the quaity of a split
- max_depth = maximum no of levels allowed in each tree
- max_features = maximum no of features considered when splitting a node
- min_samples_leaf = minimum no of samples which can be stored in a tree leaf
- min_samples_leaf = minimum no of samples necessary in a node wo cause node splitting
- n_estimators = number of trees in the ensebles

# Manual Hyperparameter Tuning

In [47]:
model = RandomForestClassifier(n_estimators=500,criterion='entropy',max_features='sqrt', min_samples_leaf=10,random_state=100).fit(X_train, y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test, predictions))
print(accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

[[357  48]
 [109 101]]
0.7447154471544716
              precision    recall  f1-score   support

           0       0.77      0.88      0.82       405
           1       0.68      0.48      0.56       210

    accuracy                           0.74       615
   macro avg       0.72      0.68      0.69       615
weighted avg       0.74      0.74      0.73       615



### Start from Randomized SearchCV because it will narrow down the results
### And the go for GridSearch

# Randomized SearchCV